In [ ]:
import torch.nn as nn

In [ ]:
import torchvision.models as models
model = models.resnet18(pretrained=True)

/home/hrudai/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hrudai/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#Freezing top k blocks of the network ( here k = 6 )
count = 0
for child in model.children():
  count+=1
  if count < 7:
    for param in child.parameters(): #To access each layer in the block
        param.requires_grad = False

In [ ]:
#To obtain the last layer nodes and modify the output layer to 10 class problem rather than 1000 class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

512

In [ ]:
import copy
best_model_wts = copy.deepcopy(model.state_dict())

In [ ]:
from PIL import Image
from numpy import asarray
import os
import numpy as np
import math
import torch
from torchvision import datasets, transforms
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader,Subset
) # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm

In [ ]:
cd drive/MyDrive/dataset_nature/inaturalist_12K/

/content/drive/MyDrive/dataset_nature/inaturalist_12K


In [ ]:
#Resizing the image to the appropriate scale that was used for pretrained model training


transform = transforms.Compose([transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),transforms.Lambda(lambda x: x/255.0)])


In [ ]:
data_dir = 'inaturalist_12K/train/'
train_dataset = datasets.ImageFolder(data_dir, transform=transform)

In [ ]:
data_dir = 'inaturalist_12K/val/'
test_dataset = datasets.ImageFolder(data_dir, transform=transform)

In [ ]:
#Loading the data and splitting into batches with transformations
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=  128, shuffle=True,num_workers = 4, pin_memory=True)

test_data = torch.utils.data.DataLoader(test_dataset,  batch_size=  128,shuffle=True, num_workers = 4,pin_memory=True)

dataloaders = {'train': train_data, 'test':test_data}
dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#Shifting the model to device (GPU if available)
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
#Declaring the optimizer for the model
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_data):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        #Back Propagation of the model
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0) #Computing the loss

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_data:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) #Computing the prediction labels for the input features
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {(correct / total) * 100:.2f}%")

100%|███████████████████████████████████████████| 79/79 [00:36<00:00,  2.16it/s]


Epoch [1/10], Loss: 1.6393


100%|███████████████████████████████████████████| 79/79 [00:39<00:00,  2.01it/s]


Epoch [2/10], Loss: 0.9497


100%|███████████████████████████████████████████| 79/79 [00:49<00:00,  1.58it/s]


...

100%|███████████████████████████████████████████| 79/79 [00:42<00:00,  1.84it/s]


Epoch [9/10], Loss: 0.3183


100%|███████████████████████████████████████████| 79/79 [00:43<00:00,  1.84it/s]

Epoch [10/10], Loss: 0.2666


Accuracy on test set: 79.20%
